## Exploring NWB format

Ref: https://pynwb.readthedocs.io/en/stable/index.html

In [1]:
# !conda install -c -y conda-forge pynwb

In [1]:
# !pip install pynwb

In [2]:
from datetime import datetime

from uuid import uuid4

import numpy as np
from dateutil import tz

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject

In [9]:
session_start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz("US/Pacific"))

nwbfile = NWBFile(
    session_description="Mouse exploring an open field",  # required
    identifier=str(uuid4()),  # required
    session_start_time=session_start_time,  # required
    session_id="session_1234",  # optional
    experimenter=[
        "Baggins, Bilbo",
    ],  # optional
    lab="Bag End Laboratory",  # optional
    institution="University of My Institution",  # optional
    experiment_description="I went on an adventure to reclaim vast treasures.",  # optional
    related_publications="DOI:10.1016/j.neuron.2016.12.011",  # optional
)
nwbfile

root pynwb.file.NWBFile at 0x140464945967552
Fields:
  experiment_description: I went on an adventure to reclaim vast treasures.
  experimenter: ['Baggins, Bilbo']
  file_create_date: [datetime.datetime(2023, 8, 15, 14, 3, 18, 820508, tzinfo=tzlocal())]
  identifier: 3fbaad4f-f7b7-416c-8d93-063629192544
  institution: University of My Institution
  lab: Bag End Laboratory
  related_publications: ['DOI:10.1016/j.neuron.2016.12.011']
  session_description: Mouse exploring an open field
  session_id: session_1234
  session_start_time: 2018-04-25 02:30:03-07:00
  timestamps_reference_time: 2018-04-25 02:30:03-07:00

## Add a subject

In [23]:
subject = Subject(
    subject_id="001",
    age="P90D",
    description="mouse 5",
    species="Mus musculus",
    sex="M",
)

nwbfile.subject = subject
subject

subject pynwb.file.Subject at 0x140464917736432
Fields:
  age: P90D
  age__reference: birth
  description: mouse 5
  sex: M
  species: Mus musculus
  subject_id: 001

## Add trials

In [13]:
nwbfile.add_trial_column(
    name="correct",
    description="whether the trial was correct",
)
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

In [14]:
nwbfile.trials.to_dataframe()

,start_time,stop_time,correct
id,,,
0,1.0,5.0,True
1,6.0,10.0,False


In [22]:
type(nwbfile.trials.to_dataframe())

pandas.core.frame.DataFrame

In [18]:
data = list(range(100, 200, 10))
timestamps = list(range(10))
time_series_with_timestamps = TimeSeries(
    name="test_timeseries",
    data=data,
    unit="m",
    timestamps=timestamps,
)
time_series_with_timestamps

test_timeseries pynwb.base.TimeSeries at 0x140464915172560
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: no description
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: [0 1 2 3 4 5 6 7 8 9]
  timestamps_unit: seconds
  unit: m

In [19]:
nwbfile.add_epoch(
    start_time=2.0,
    stop_time=4.0,
    tags=["first", "example"],
    timeseries=[time_series_with_timestamps],
)

nwbfile.add_epoch(
    start_time=6.0,
    stop_time=8.0,
    tags=["second", "example"],
    timeseries=[time_series_with_timestamps],
)

In [20]:
nwbfile.epochs.to_dataframe()

,start_time,stop_time,tags,timeseries
id,,,,
0,2.0,4.0,"[first, example]","[(2, 2, test_timeseries pynwb.base.TimeSeries ..."
1,6.0,8.0,"[second, example]","[(6, 2, test_timeseries pynwb.base.TimeSeries ..."


## Save

In [ ]:
io = NWBHDF5IO("basics_tutorial.nwb", mode="a")

## Load